In [1]:
import cv2
import random
import time
from datetime import datetime
import matplotlib.pyplot as plt
import PIL.Image 
from PIL import Image 
import pickle
import dlib
from tkinter import *
from tkinter import ttk  
import tkinter as tk
import os
from os import listdir
from mtcnn.mtcnn import MTCNN
import win32com.client as win
import numpy as np
from numpy import load
from numpy import asarray
from numpy import expand_dims
from numpy import savez_compressed
from numpy import reshape
from numpy import max
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow import keras
from keras_facenet import FaceNet
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
"""
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.models import model_from_json
from keras.models import load_model
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Conv2D,Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.applications.imagenet_utils import preprocess_input
"""
from mtcnn_cv2 import MTCNN

Using TensorFlow backend.


In [2]:
#detect + take pic
def new_user():
    image_path = 'Data/images/'
    detector = MTCNN()

    cap = cv2.VideoCapture(0)
    name = input('enter user :')
    count = 0
    id_rand = random.sample(range(1,10000),1)
    myList = os.listdir(image_path)

    for line in myList:
        Type = line.split(".")
        split = os.path.splitext(line)
        if len(Type) > 2:
            y = Type[1]
        elif len(split) >= 2:
            y = split[1]
        if id_rand == y:
            id_rand = random.sample(range(1,10000),1)

    while cap.isOpened():
        ret, frame = cap.read()
        new_path = image_path+str(name)+"."+ str(id_rand)
        col = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pixels = np.asarray(col)  
        detections = detector.detect_faces(pixels)
        for detection in detections:
            score = detection["confidence"]
            if score > 0.90:
                x, y, w, h = detection["box"]
                count+=1 
                if not os.path.exists(new_path):
                    os.makedirs(new_path)
                cv2.imwrite(new_path+"/"+str(name)+"."+str(count)+ ".jpg", col[y-20:y+h+20, x-20:x+w+20])    
                cv2.waitKey(100)
                if count == 20:
                    cap.release()
                    cv2.destroyAllWindows()

        cv2.imshow('cam',frame)
        cv2.destroyAllWindows()

In [3]:
#_training
#Method to extract Face
def extract_image(image):
    img1 = PIL.Image.open(image)            #open the image
    img1 = img1.convert('RGB')          #convert the image to RGB format 
    pixels = asarray(img1)              #convert the image to numpy array
    detector = MTCNN()                  #assign the MTCNN detector
    f = detector.detect_faces(pixels)
    #fetching the (x,y)co-ordinate and (width-->w, height-->h) of the image
    x1,y1,w,h = f[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2 = abs(x1+w)
    y2 = abs(y1+h)
    #locate the co-ordinates of face in the image
    store_face = pixels[y1:y2,x1:x2]  
    image1 = PIL.Image.fromarray(store_face,'RGB')    #convert the numpy array to object
    image1 = image1.resize((160,160))             #resize the image
    face_array = asarray(image1)                  #image to array
    return face_array

#Method to fetch the face
def load_faces(directory):
    face = []
    i=1
    for filename in listdir(directory):
        if filename.endswith("png") or filename.endswith("jpg") or filename.endswith("jpeg"): 
            path = directory + filename
            faces = extract_image(path)
            face.append(faces)
    return face

#Method to get the array of face data(trainX) and it's labels(trainY)
def load_dataset(directory):
    x, y = [],[]
    i=1
    for subdir in listdir(directory):
        path = directory + subdir + '/'
        #load all faces in subdirectory
        faces = load_faces(path)
        #create labels
        labels = [subdir for _ in range(len(faces))]
        #summarize
        print("%d There are %d images in the class %s:"%(i,len(faces),subdir))
        x.extend(faces)
        y.extend(labels)
        i=i+1
    return asarray(x),asarray(y)  
def training(path):
    #load the datasets
    trainX,trainY = load_dataset(path)
    try:
        print('\033[1m' + str(trainX.shape + trainY.shape))
    except:
        print('\033[1m' + 'problem with shape')
    #compress the data
    savez_compressed('data/trainer/facenet-mtcnn-dataset.npz',trainX,trainY)

In [4]:
#embedding
def embedding():
    #load the compressed dataset and facenet keras model
    data = load('data/trainer/facenet-mtcnn-dataset.npz')
    trainx, trainy = data['arr_0'],data['arr_1']
    print(trainx.shape, trainy.shape)

    model = FaceNet()
    #get the face embeddings
    new_trainx = list()
    embeddings = model.embeddings(trainx)
    new_trainx.append(embeddings)
    new_trainx = asarray(new_trainx)             #convert the embeddings into numpy array
    print(new_trainx.shape, trainy.shape)

    #compress the 128 embeddings of each face 
    savez_compressed('data/trainer/facenet-mtcnn-embeddings.npz',new_trainx,trainy)

In [5]:
#recognizer
def recognize():
    detector = MTCNN()
    #detector= FaceRecognition()
    font = cv2.FONT_HERSHEY_SIMPLEX

    cap = cv2.VideoCapture(0)
    #cap.set(3, 1024) # set video widht
    #cap.set(4, 720)
    while cap.isOpened():
        ret, frame = cap.read()
        detector = MTCNN()
        model = FaceNet()
        img1 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pixels = np.asarray(img1)
        detections = detector.detect_faces(pixels)
        for detection in detections:
            x, y, w, h = detection["box"]
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),1)
            store_face = pixels[y:y+h,x:x+w]
            image1 = PIL.Image.fromarray(store_face,'RGB')
            store_f = image1.resize((160,160), PIL.Image.ANTIALIAS)
            face_array = np.asarray(store_f)
            #face_array = face_array.reshape(-1,160,160,3)     
            face_array = np.expand_dims(face_array, axis = 0)
            #print("Input test data shape: ",face_array.shape)

            new_testx = list()
            embeddings = model.embeddings(face_array)
            new_testx.append(embeddings)
            new_testx = asarray(new_testx)  
            #print("Input test embedding shape: ",new_testx.shape)

            data1 = load('data/trainer/facenet-mtcnn-dataset.npz')
            train_x,train_y = data1['arr_0'],data1['arr_1']

            data = load('data/trainer/facenet-mtcnn-embeddings.npz')
            trainx,trainy= data['arr_0'],data['arr_1']
            #print("Loaded data: Train=%d , Test=%d"%(trainx.shape[0],new_testx.shape[0]))      

            nsamples, nx, ny = trainx.shape
            d2_train_x = trainx.reshape((nsamples*nx,ny))
            nsamples, nx, ny = new_testx.shape
            d2_train_n = new_testx.reshape((nsamples*nx,ny))
            #print(d2_train_x.shape, d2_train_n.shape)

            #normalize the input data
            in_encode = Normalizer(norm='l2')
            trainx = in_encode.transform(d2_train_x)
            new_testx = in_encode.transform(d2_train_n)

            #create a label vector
            new_testy = trainy 
            out_encode = LabelEncoder()
            out_encode.fit(trainy)
            trainy = out_encode.transform(trainy)
            new_testy = out_encode.transform(new_testy)

            #define svm classifier model 
            model =SVC(kernel='linear', probability=True)
            model.fit(trainx,trainy)

            #predict
            predict_train = model.predict(trainx)
            predict_test = model.predict(new_testx)

            #get the confidence score
            probability = model.predict_proba(new_testx)
            confidence = max(probability)

            #Accuracy
            acc_train = accuracy_score(trainy,predict_train)

            #display
            trainy_list = list(trainy)
            p=int(predict_test)
            if p in trainy_list:
                val = trainy_list.index(p)

            predict_test = out_encode.inverse_transform(predict_test)
            #cv2.putText(frame, predict_test, (x, y), font, 0.5,(255,255,255),1,cv2.LINE_AA)
            #cv2.putText(frame, train_x[val], (x, y), font, 0.5,(255,255,255),1,cv2.LINE_AA)
            trainy = out_encode.inverse_transform(trainy)
            #cv2.putText(frame, trainy[val], (x, y), font, 0.5,(255,255,255),1,cv2.LINE_AA)

            name = trainy[val].split(".")[0]
            print(confidence)
            print(name)
            speak=win.Dispatch("SAPI.spVoice")
            speak.rate=-1
            #speak.speak(name)
            color = (127,63,0)
            cv2.putText(frame,  name+'/'+str("{:.2f}".format(confidence)), (int(x+w+15), int(y-64)), font, 1, color, 2)
            #cv2.putText(frame,  name, (x, y), font, 1, color, 2)
            #connect face and text
            cv2.line(frame,(x+w, y-64),(x+w-25, y-64),color,1)
            cv2.line(frame,(int(x+w/2),y),(x+w-25,y-64),color,1)
            
            with open('data/attendance/Attendance_mtcnn+facenet.csv','r+') as f:
                myDataList = f.readlines()
                nameList = []
                for line in myDataList:
                    entry = line.split(',')
                    nameList.append(entry[0])
                if name not in nameList:
                    now = datetime.now()
                    dtString = now.strftime('%H:%M:%S')
                    f.writelines(f'\n{name},{dtString}')
                    speak.speak(name)

        cv2.imshow('cam', frame)
        #plt.imshow(frame)
        key = cv2.waitKey(1)
        if key == 13:
            break
    cap.release()
    cv2.destroyAllWindows()

In [6]:
path ="data/images/"
def F_popup():
    pop_up = tk.Tk()
    pop_up.wm_title('Facial Recognition')
    pop_up.geometry("600x200")
    pop_up.grid_columnconfigure(0, weight=1)
    pop_up.grid_rowconfigure(0, weight=1)
    pop_up.grid_columnconfigure(1, weight=1)
    pop_up.grid_rowconfigure(1, weight=1)
    
    b1 = ttk.Button(pop_up, text='Train model', command=lambda:[training(path)])
    b1.grid(row=0, column=0, sticky=NSEW)
    b2 = ttk.Button(pop_up, text='embedding', command=embedding)
    b2.grid(row=0, column=1, sticky=NSEW)
    b3 = ttk.Button(pop_up, text='Facial recognition', command=recognize)
    b3.grid(row=1, column=0, sticky=NSEW)
    b4 = ttk.Button(pop_up, text='Close', command=pop_up.destroy)
    b4.grid(row=1, column=1, sticky=NSEW)
    pop_up.mainloop()
    
pop_up = tk.Tk()
pop_up.wm_title('Facial Recognition')
pop_up.geometry("600x100")
b1 = ttk.Button(pop_up, text='New users', command=new_user)
b1.pack(side='left',pady=10,fill='both', expand=True)
b2 = ttk.Button(pop_up, text='Continue', command = lambda:[pop_up.destroy(),F_popup()])
b2.pack(side='right', pady=10,fill='both', expand=True)
pop_up.mainloop()

enter user : wassim


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-2-7941c5bb18c5>", line 35, in new_user
    cv2.imwrite(new_path+"/"+str(name)+"."+str(count)+ ".jpg", col[y-20:y+h+20, x-20:x+w+20])
cv2.error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-71670poj\opencv\modules\imgcodecs\src\loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'



1 There are 1 images in the class Aaron_Guiel.[80711]:


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-6-58709c103410>", line 11, in <lambda>
    b1 = ttk.Button(pop_up, text='Train model', command=lambda:[training(path)])
  File "<ipython-input-3-b526e8c95a94>", line 50, in training
    trainX,trainY = load_dataset(path)
  File "<ipython-input-3-b526e8c95a94>", line 39, in load_dataset
    faces = load_faces(path)
  File "<ipython-input-3-b526e8c95a94>", line 28, in load_faces
    faces = extract_image(path)
  File "<ipython-input-3-b526e8c95a94>", line 10, in extract_image
    x1,y1,w,h = f[0]['box']
IndexError: list index out of range


1 There are 1 images in the class Aaron_Guiel.[80711]:


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-6-58709c103410>", line 11, in <lambda>
    b1 = ttk.Button(pop_up, text='Train model', command=lambda:[training(path)])
  File "<ipython-input-3-b526e8c95a94>", line 50, in training
    trainX,trainY = load_dataset(path)
  File "<ipython-input-3-b526e8c95a94>", line 39, in load_dataset
    faces = load_faces(path)
  File "<ipython-input-3-b526e8c95a94>", line 28, in load_faces
    faces = extract_image(path)
  File "<ipython-input-3-b526e8c95a94>", line 10, in extract_image
    x1,y1,w,h = f[0]['box']
IndexError: list index out of range
